In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch06. RNN기반의 Seq2Seq(스마트번역기)</font></b>

- Google Neural Machine Translation(GNMT)
- RNN 기반의 Sequence to Sequence
- 인코더(입력) / 디코더(모범출력) 연결구조
- 응용분야 : 자연어

# 1. 패키지 및 하이퍼파라미터

In [77]:
import numpy as np
import pandas as pd

from time import time

from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate, Dense, Input, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# 하이퍼파라미터

MY_HIDDEN = 128
MY_EPOCH  = 500

# 2. 학습 데이터

In [11]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터프레임을 list로 변환
print(eng_kor[:3])
print('영-한 번역 데이터 갯수 :', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영-한 번역 데이터 갯수 : 110


# 3. 영어알파벳과 한글문자 리스트 만들기

In [20]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
k_ch[:3]

['가', '각', '간']

In [23]:
k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
k_alpha == k_ch # 순서와 내용이 같은지 여부. 둘 중 뭘 써도 상관없음

True

In [24]:
# k_alpha == k_ch는 순서가 같아야함. 내용만 비교하고 싶을때 쓰는 함수는

from collections import Counter
list1 = ['가', '나', '다']
list2 = ['다', '가', '나']
print(list1==list2)
print(Counter(list1)==Counter(list2))

False
True


In [28]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 :', alpha)
alpha_total_size = len(alpha) # 171 : (원핫인코딩 사이즈)
print('전체 알파벳 갯수(원핫인코딩 사이즈) :', alpha_total_size)

영어와 한글 알파벳 : ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩 사이즈) : 171


# 4. 문자당 num을 갖는 dict

In [34]:
char_to_num = {}
# for i, c in enumerate(alpha):
#     # print(i,c)
#     char_to_num[c] = i
# print(char_to_num)

In [35]:
# 리스트컨프리핸션을 {}로도 쓸 수 있음

char_to_num = {c:i for i,c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [37]:
# 숫자 → 문자 / 문자 → 숫자

print('숫자 → 문자 :', alpha[5])
print('문자 → 숫자 :', char_to_num['c'])

숫자 → 문자 : c
문자 → 숫자 : 5


In [44]:
data = eng_kor[0]
print('원데이터 :', data)
print('인코더 입력 :', [char_to_num[ch] for ch in data[0]])
print('디코더 입력 :', [char_to_num[ch] for ch in 'S' + data[1]])
print('디코더 출력 :', [char_to_num[ch] for ch in data[1] + 'E'])

원데이터 : ['cold', '감기']
인코더 입력 : [5, 17, 14, 6]
디코더 입력 : [0, 32, 46]
디코더 출력 : [32, 46, 1]


In [46]:
# 원핫인코딩 방법 1 : pd.get_dummies([5, 3, 7]) - 이 코드에서는 적용 불가

pd.get_dummies([5, 3, 7])

,3,5,7
0,0,1,0
1,1,0,0
2,0,0,1


In [47]:
# 원핫인코딩 방법 2 : to_categorical([5, 3, 7], num_classes=10)

to_categorical([[5, 3, 7]], num_classes=10)

array([[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]], dtype=float32)

In [49]:
# 원핫인코딩 방법 3 : np.eye(10) 단위행렬 만드는것

np.eye(10)[[5, 3, 7]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

# 5. 인코더 입력, 디코더 입력, 디코더 출력

- 인코더입력, 디코더입력(원핫인코딩O)
- 디코더출력(원핫인코딩 X)

In [62]:
def encoding(eng_kor=eng_kor):
    '인코더입력, 디코더입력, 디코더출력 데이터를 return'
    enc_in  = [] # 인코더입력
    dec_in  = [] # 디코더입력
    dec_out = [] # 디코더 출력
    for data in eng_kor:
        
        # 인코더 입력 데이터(영어 → 숫자 → 원핫인코딩)
        eng = [char_to_num[ch] for ch in data[0]]
        # print(data[0], ':', eng)
        # eng_one = to_categorical(eng, num_classes=alpha_total_size) # alpha_total_size : 171
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)
        
        # 디코더 입력 데이터(S한글 → 숫자 → 원핫인코딩)
        kor = [char_to_num[ch] for ch in 'S' + data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        
        # 디코더 출력 (한글E → 숫자)
        kor = [char_to_num[ch] for ch in data[1] + 'E']
        dec_out.append(kor)
    
    # 인공신경에 넣을 데이터이므로 numpy 배열로 전환
    enc_in  = np.array(enc_in)
    dec_in  = np.array(dec_in)
    dec_out = np.array(dec_out)
    # print(enc_in.shape, dec_in.shape, dec_out.shape)
    return enc_in, dec_in, dec_out 
        
sample = [['wood','나무'], ['word','단어']]

In [64]:
# RNN 분류분석 : 타겟변수를 원핫인코딩(시스템에게 원핫인코딩을 의뢰)

X_enc, X_dec, y_dec = encoding(sample)
X_enc.shape, X_dec.shape, y_dec.shape

((2, 4, 171), (2, 3, 171), (2, 3))

## 축 증가

In [67]:
# 방법 1
y_dec.reshape(2,3,1) 

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [68]:
# 방법 2
np.expand_dims(y_dec, axis=-1)

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [69]:
# 방법 3 : 이미지에서 많이 씀
y_dec[..., np.newaxis]

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [70]:
# 방법 4 : 거의 안씀
y_dec[:,:,None]

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

# 6. 전체 번역 데이터(독립변수, 타겟변수)

In [74]:
X_enc, X_dec, y_dec = encoding(eng_kor)
Y_dec = np.expand_dims(y_dec, axis=-1)
# Y_dec = y_dec.reshape(-1, 3, 1)

X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

# 7. 모델구현

In [76]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))

# 윗출력, state_h, state_c. 옆으로 가는건 변수로 받아야함. DEC_IN은 위로 올라가는거라 굳이 변수로 받을 필요없음
_, state_h, state_c = LSTM(units=MY_HIDDEN,
                           # return_sequences=False 윗출력 안받음. 기본이 False
                           return_state=True, # state값을 옆으로 전달하는 역할
                           )(ENC_IN)

# 인코더와 디코더를 연결할 link
link = [state_h, state_c]

# 디코더 구현 : return_sequences=True 위로 올라가는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
               # return_state=False 기본이 False
               return_sequences=True)(DEC_IN, initial_state=link)

# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size, 
                activation='softmax')(DEC_MID)

# 모델객체
model = Model(inputs=[ENC_IN, DEC_IN], outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 128),        153600      ['input_3[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 8. 학습과정 설정 및 학습하기

In [78]:
# 학습과정 설정하기
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

start = time()

# 콜백객체
earlystopping = EarlyStopping(monitor='accuracy', # 우리는 지금 validation data가 없기 때문에 안쓰면 기본인 val_loss가 들어가서 꼭 써줘야함
                              patience=20)

# 학습하기
hist = model.fit([X_enc, X_dec], Y_dec,
                 epochs=MY_EPOCH,
                 verbose=1,
                 callbacks=[earlystopping])
end = time()
print(f'학습시간 : {end-start}')

Epoch 1/500
4/4 [==============================] - 2s 7ms/step - loss: 5.1185 - acc: 0.2121
Epoch 2/500
4/4 [==============================] - 0s 8ms/step - loss: 4.9646 - acc: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 8ms/step - loss: 4.1686 - acc: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 3.4720 - acc: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 7ms/step - loss: 3.4198 - acc: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3860 - acc: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 8ms/step - loss: 3.3486 - acc: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3220 - acc: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 8ms/step - loss: 3.3019 - acc: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2790 - acc: 0.3333
Epoch 11/500
4/4 [==============================] - 0s 8ms/step - loss: 3.2504 

4/4 [==============================] - 0s 7ms/step - loss: 2.9539 - acc: 0.3667
Epoch 30/500
4/4 [==============================] - 0s 7ms/step - loss: 2.9373 - acc: 0.3727
Epoch 31/500
4/4 [==============================] - 0s 7ms/step - loss: 2.9240 - acc: 0.3758
Epoch 32/500
4/4 [==============================] - 0s 7ms/step - loss: 2.9124 - acc: 0.3727
Epoch 33/500
4/4 [==============================] - 0s 7ms/step - loss: 2.8830 - acc: 0.3758
Epoch 34/500
4/4 [==============================] - 0s 7ms/step - loss: 2.8618 - acc: 0.3758
Epoch 35/500
4/4 [==============================] - 0s 9ms/step - loss: 2.8492 - acc: 0.3727
Epoch 36/500
4/4 [==============================] - 0s 7ms/step - loss: 2.8468 - acc: 0.3758
Epoch 37/500
4/4 [==============================] - 0s 7ms/step - loss: 2.8024 - acc: 0.3909
Epoch 38/500
4/4 [==============================] - 0s 7ms/step - loss: 2.7792 - acc: 0.3939
Epoch 39/500
4/4 [==============================] - 0s 7ms/step - loss: 2.7525 - ac

4/4 [==============================] - 0s 7ms/step - loss: 2.2115 - acc: 0.4576
Epoch 58/500
4/4 [==============================] - 0s 8ms/step - loss: 2.3231 - acc: 0.4485
Epoch 59/500
4/4 [==============================] - 0s 7ms/step - loss: 2.1476 - acc: 0.4576
Epoch 60/500
4/4 [==============================] - 0s 7ms/step - loss: 2.1133 - acc: 0.4818
Epoch 61/500
4/4 [==============================] - 0s 7ms/step - loss: 2.0826 - acc: 0.4970
Epoch 62/500
4/4 [==============================] - 0s 7ms/step - loss: 2.0561 - acc: 0.4939
Epoch 63/500
4/4 [==============================] - 0s 9ms/step - loss: 2.0122 - acc: 0.5182
Epoch 64/500
4/4 [==============================] - 0s 7ms/step - loss: 1.9938 - acc: 0.5212
Epoch 65/500
4/4 [==============================] - 0s 7ms/step - loss: 2.0001 - acc: 0.5061
Epoch 66/500
4/4 [==============================] - 0s 8ms/step - loss: 1.9203 - acc: 0.5364
Epoch 67/500
4/4 [==============================] - 0s 7ms/step - loss: 1.8682 - ac

4/4 [==============================] - 0s 7ms/step - loss: 1.1990 - acc: 0.8212
Epoch 86/500
4/4 [==============================] - 0s 7ms/step - loss: 1.1586 - acc: 0.8576
Epoch 87/500
4/4 [==============================] - 0s 7ms/step - loss: 1.1216 - acc: 0.8606
Epoch 88/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0957 - acc: 0.8758
Epoch 89/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0705 - acc: 0.8818
Epoch 90/500
4/4 [==============================] - 0s 8ms/step - loss: 1.0452 - acc: 0.8727
Epoch 91/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0244 - acc: 0.8818
Epoch 92/500
4/4 [==============================] - 0s 7ms/step - loss: 0.9673 - acc: 0.8970
Epoch 93/500
4/4 [==============================] - 0s 7ms/step - loss: 0.9388 - acc: 0.9000
Epoch 94/500
4/4 [==============================] - 0s 7ms/step - loss: 0.9170 - acc: 0.9091
Epoch 95/500
4/4 [==============================] - 0s 7ms/step - loss: 0.8813 - ac

4/4 [==============================] - 0s 8ms/step - loss: 0.4497 - acc: 0.9636
Epoch 114/500
4/4 [==============================] - 0s 7ms/step - loss: 0.4368 - acc: 0.9818
Epoch 115/500
4/4 [==============================] - 0s 6ms/step - loss: 0.4177 - acc: 0.9758
Epoch 116/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3974 - acc: 0.9818
Epoch 117/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3879 - acc: 0.9727
Epoch 118/500
4/4 [==============================] - 0s 7ms/step - loss: 0.3705 - acc: 0.9848
Epoch 119/500
4/4 [==============================] - 0s 9ms/step - loss: 0.3559 - acc: 0.9848
Epoch 120/500
4/4 [==============================] - 0s 6ms/step - loss: 0.3412 - acc: 0.9909
Epoch 121/500
4/4 [==============================] - 0s 6ms/step - loss: 0.3236 - acc: 0.9909
Epoch 122/500
4/4 [==============================] - 0s 6ms/step - loss: 0.3136 - acc: 0.9848
Epoch 123/500
4/4 [==============================] - 0s 7ms/step - loss: 0

...

4/4 [==============================] - 0s 7ms/step - loss: 1.4970e-06 - acc: 1.0000
Epoch 391/500
4/4 [==============================] - 0s 6ms/step - loss: 1.4793e-06 - acc: 1.0000
Epoch 392/500
4/4 [==============================] - 0s 8ms/step - loss: 1.4627e-06 - acc: 1.0000
Epoch 393/500
4/4 [==============================] - 0s 7ms/step - loss: 1.4330e-06 - acc: 1.0000
Epoch 394/500
4/4 [==============================] - 0s 7ms/step - loss: 1.4291e-06 - acc: 1.0000
Epoch 395/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3991e-06 - acc: 1.0000
Epoch 396/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3807e-06 - acc: 1.0000
Epoch 397/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3684e-06 - acc: 1.0000
Epoch 398/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3489e-06 - acc: 1.0000
Epoch 399/500
4/4 [==============================] - 0s 7ms/step - loss: 1.3294e-06 - acc: 1.0000
Epoch 400/500
4/4 [===============

4/4 [==============================] - 0s 7ms/step - loss: 1.0928e-06 - acc: 1.0000
Epoch 418/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0859e-06 - acc: 1.0000
Epoch 419/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0628e-06 - acc: 1.0000
Epoch 420/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0577e-06 - acc: 1.0000
Epoch 421/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0527e-06 - acc: 1.0000
Epoch 422/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0400e-06 - acc: 1.0000
Epoch 423/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0317e-06 - acc: 1.0000
Epoch 424/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0133e-06 - acc: 1.0000
Epoch 425/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0046e-06 - acc: 1.0000
Epoch 426/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0039e-06 - acc: 1.0000
Epoch 427/500
4/4 [===============

4/4 [==============================] - 0s 7ms/step - loss: 8.4494e-07 - acc: 1.0000
Epoch 446/500
4/4 [==============================] - 0s 8ms/step - loss: 8.3446e-07 - acc: 1.0000
Epoch 447/500
4/4 [==============================] - 0s 8ms/step - loss: 8.2977e-07 - acc: 1.0000
Epoch 448/500
4/4 [==============================] - 0s 7ms/step - loss: 8.1749e-07 - acc: 1.0000
Epoch 449/500
4/4 [==============================] - 0s 7ms/step - loss: 8.1749e-07 - acc: 1.0000
Epoch 450/500
4/4 [==============================] - 0s 7ms/step - loss: 8.1315e-07 - acc: 1.0000
Epoch 451/500
4/4 [==============================] - 0s 7ms/step - loss: 8.0665e-07 - acc: 1.0000
Epoch 452/500
4/4 [==============================] - 0s 8ms/step - loss: 8.0557e-07 - acc: 1.0000
Epoch 453/500
4/4 [==============================] - 0s 8ms/step - loss: 7.9292e-07 - acc: 1.0000
Epoch 454/500
4/4 [==============================] - 0s 8ms/step - loss: 7.8859e-07 - acc: 1.0000
Epoch 455/500
4/4 [===============

4/4 [==============================] - 0s 8ms/step - loss: 6.9250e-07 - acc: 1.0000
Epoch 473/500
4/4 [==============================] - 0s 7ms/step - loss: 6.8708e-07 - acc: 1.0000
Epoch 474/500
4/4 [==============================] - 0s 7ms/step - loss: 6.8419e-07 - acc: 1.0000
Epoch 475/500
4/4 [==============================] - 0s 8ms/step - loss: 6.7624e-07 - acc: 1.0000
Epoch 476/500
4/4 [==============================] - 0s 8ms/step - loss: 6.6938e-07 - acc: 1.0000
Epoch 477/500
4/4 [==============================] - 0s 7ms/step - loss: 6.6324e-07 - acc: 1.0000
Epoch 478/500
4/4 [==============================] - 0s 8ms/step - loss: 6.6360e-07 - acc: 1.0000
Epoch 479/500
4/4 [==============================] - 0s 8ms/step - loss: 6.6071e-07 - acc: 1.0000
Epoch 480/500
4/4 [==============================] - 0s 9ms/step - loss: 6.5493e-07 - acc: 1.0000
Epoch 481/500
4/4 [==============================] - 0s 8ms/step - loss: 6.4879e-07 - acc: 1.0000
Epoch 482/500
4/4 [===============

4/4 [==============================] - 0s 7ms/step - loss: 5.7943e-07 - acc: 1.0000
학습시간 : 18.302756786346436


In [79]:
model.save('model/seq2seq.h5')